# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.model import Model
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, AciWebservice
import json
import numpy as np
from azureml.automl.core.shared import constants

import joblib 
import azureml.automl
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.pipeline.core import Pipeline
import pickle
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import Webservice, AciWebservice

import requests
import json



from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split



from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-136423
aml-quickstarts-136423
southcentralus
5a4ab2ba-6c51-4805-8155-58759ad589d8


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'ml-capstone-1'
project_folder = './capstone-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ml-capstone-1,quick-starts-ws-136423,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
#Create or Attach an AmlCompute cluster

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Creating
Succeeded...............................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "wine-quality"
description_text = " Datasets from wine quality of red wine the Portuguese Vinho Verde wine"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1595.000000,1597.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.838871,46.428929,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.423696,32.897570,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [6]:
#Review Dataset
dataset.take(5).to_pandas_dataframe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17,60,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5


In [7]:
train, test = train_test_split(df, test_size=0.2, random_state=1 )

if "training" not in os.listdir():
    os.mkdir("./training")

if "models" not in os.listdir():
    os.mkdir("./models")
if "outputs" not in os.listdir():
    os.mkdir("./outputs")

train.to_csv("training/train_data.csv", index = False)

data_store = ws.get_default_datastore()
data_store.upload(src_dir = "./training", target_path = 'capstone-project', overwrite = True,  show_progress = True)

Uploading an estimated of 1 files
Uploading ./training/train_data.csv
Uploaded ./training/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_f5496765e08d48b1941b52f88bc88a0d

In [8]:
train_data = Dataset.Tabular.from_delimited_files(path=data_store.path("capstone-project/train_data.csv"))

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [9]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'    
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    
    compute_target = compute_target,
    task = "classification",
    training_data = train_data,
    label_column_name = "quality",   
    path = project_folder,
    enable_early_stopping= True,
    featurization= 'auto',
    debug_log = "automl_errors.log",
    **automl_settings 
)

In [10]:
# TODO: Submit your experiment
from azureml.widgets import RunDetails

automl_run = experiment.submit(automl_config, show_output=True)
automl_run.wait_for_completion(show_output=True)

Running on remote.
No run_configuration provided, running on auto-ml with default configuration
Running on remote compute: auto-ml
Parent Run ID: AutoML_d60e34fe-f18d-46a4-8dfe-77b12dca0a2f

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing det


****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|9                                |3                                |1279                                  |
+---------------------------------+-----------------------------

{'runId': 'AutoML_d60e34fe-f18d-46a4-8dfe-77b12dca0a2f',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2021-01-28T20:07:32.531814Z',
 'endTimeUtc': '2021-01-28T20:35:46.288451Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'auto-ml',
  'AMLSettingsJsonString': '{"path":null,"name":"ml-capstone-1","subscription_id":"5a4ab2ba-6c51-4805-8155-58759ad589d8","resource_group":"aml-quickstarts-136423","workspace_name":"quick-starts-ws-136423","region":"southcentralus","compute_target":"auto-ml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":null,"featuriza

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [17]:
#View more details from running cell
from azureml.widgets import RunDetails

RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------

{'runId': 'AutoML_f2600643-baea-4a05-8786-d99bb8e114b5',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2021-01-25T21:07:42.635533Z',
 'endTimeUtc': '2021-01-25T21:33:03.719469Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'auto-ml',
  'AMLSettingsJsonString': '{"path":null,"name":"ml-capstone-1","subscription_id":"d7f39349-a66b-446e-aba6-0053c2cf1c11","resource_group":"aml-quickstarts-136033","workspace_name":"quick-starts-ws-136033","region":"southcentralus","compute_target":"auto-ml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":null,"featuriza

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
#TODO: Save the best model
# Retrieve and save your best automl model.
#Validate highest performance Model


best_model, fitted_model = automl_run.get_output()
print(best_model)

Run(Experiment: ml-capstone-1,
Id: AutoML_d60e34fe-f18d-46a4-8dfe-77b12dca0a2f_38,
Type: azureml.scriptrun,
Status: Completed)


In [12]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_split=0.19736842105263158,
                                                                                                  min_weight_fraction_leaf=0.0,
                                                                                                  n_estimators=50,
            

In [13]:
automl_run.get_metrics('accuracy')

{'accuracy': 0.666149904893844}

In [14]:
print(automl_run.get_portal_url())

https://ml.azure.com/experiments/ml-capstone-1/runs/AutoML_d60e34fe-f18d-46a4-8dfe-77b12dca0a2f?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-136423/workspaces/quick-starts-ws-136423


In [15]:
#SAVE model

best_model.download_file('outputs/model.pkl', 'outputs/automl_wine.pkl')
best_model.download_file('outputs/scoring_file_v_1_0_0.py', 'outputs/score_aml.py')
best_model.download_file('automl_driver.py', 'outputs/automl_driver.py')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
#Register a model

model = Model.register(workspace = ws,
                        model_path ="outputs/automl_wine.pkl",
                        model_name = "automl_wine")

#model = Model.register(workspace =ws, model_path= './', model_name='best_automl_run.pkl')
best_model.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
#print(model.name, model.id, model.version, sep='\t')

Registering model automl_wine


In [17]:

# Define Deployment
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2,
                                                tags = {'name':'deployment'},
                                   description='Quality Portuguese red wine',
                                              enable_app_insights=True) 



In [18]:
# Create the environment

env = Environment.from_conda_specification(name="myenv",file_path="myenv.yml")
inferencefinal = InferenceConfig(entry_script='score.py',environment=env)


f = open("envnew.yml", "w")
f.write(env.python.conda_dependencies.serialize_to_string())
f.close()

print("packages", env.python.conda_dependencies.serialize_to_string())

packages # Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-train-automl-runtime==1.20.0.post1
  - inference-schema
  - azureml-interpret==1.20.0
  - azureml-defaults==1.20.0
- numpy>=1.16.0,<1.19.0
- pandas==0.25.1
- scikit-learn==0.22.1
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- psutil>=5.2.2,<6.0.0
channels:
- anaconda
- conda-forge



TODO: In the cell below, send a request to the web service you deployed to test it.

In [19]:
service = Model.deploy(workspace=ws,name='demo-deploy', models=[model], inference_config=inferencefinal, deployment_config=aci_config)


In [20]:
service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...................................................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [21]:
print("scoring URI: " + service.scoring_uri)

scoring URI: http://6b049cf7-78dc-4681-99c6-200100f86197.southcentralus.azurecontainer.io/score


TODO: In the cell below, print the logs of the web service and delete the service

In [22]:
# URL for the web service, should be similar to:

scoring_uri = 'http://6b049cf7-78dc-4681-99c6-200100f86197.southcentralus.azurecontainer.io/score'

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
            "fixed acidity":7.4,
            "volatile acidity":0.70,
            "citric acid":0.00,
            "residual sugar":1.9,
            "chlorides":0.098,
            "free sulfur dioxide":11,
            "total sulfur dioxide":34,
            "density":0.9978,
            "pH":3.51,
            "sulphates":0.55,
            "alcohol":9.4,
            
          },
          {
            "fixed acidity":7.4,
            "volatile acidity":0.88,
            "citric acid":0.00,
            "residual sugar":2.6,
            "chlorides":0.098,
            "free sulfur dioxide":25,
            "total sulfur dioxide":67,
            "density":0.9968,
            "pH":3.20,
            "sulphates":0.68,
            "alcohol":9.8,
           
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}


# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())


[5, 5]


In [23]:
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

2021-01-28T21:09:48,516891925+00:00 - rsyslog/run 
2021-01-28T21:09:48,518871799+00:00 - iot-server/run 
2021-01-28T21:09:48,520171482+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-28T21:09:48,518255007+00:00 - gunicorn/run 
rsyslogd

TODO: In the cell below, print the logs of the web service and delete the service

In [24]:
## print request status_code 
print(resp.status_code)
print(resp.elapsed)
print(resp.json())

200
0:00:00.202965
[5, 5]


In [32]:
#Delete Service
service.delete()

In [33]:
#To save money on Azure Cost, delete compute cluster
compute_target.delete()